In [ ]:
#
# LLM with QnA and Chain
#
!pip install youtube-transcript-api # YouTube lib to get transcript
!pip install pytube                 # YouTube lib
!pip install openai                 # OpenAI GPT
!pip install transformers           # Used to know the token size
!pip install python-dotenv
!pip install chromadb==0.3.2
!pip install langchain==0.0.103

In [ ]:
!git clone https://github.com/hwchase17/langchain.git

In [ ]:
from dotenv import dotenv_values
api_keys = dotenv_values('keys.txt')

import os
os.environ['OPENAI_API_KEY'] = api_keys['OPENAI_API_KEY']
os.environ['SERPER_API_KEY'] = api_keys['SERPER_API_KEY']

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

In [ ]:
from langchain.document_loaders.unstructured import UnstructuredFileLoader
from langchain.document_loaders import YoutubeLoader

In [ ]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=FtyKgybYkaE", add_video_info=False)
result = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

docs = text_splitter.split_documents(result)
docs

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
db = Chroma.from_documents(docs, embeddings, persist_directory='db')

In [ ]:
query = "What's the Devoxx Belgium 2022 movie? Only answer with the movie name nothing else."

In [ ]:
search_results = db.similarity_search(query)

In [ ]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=search_results, question=query)

In [ ]:
query = "When is the next Devoxx Belgium?"

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

tools = load_tools(["google-serper"], llm=llm)

In [ ]:
from langchain.chains import LLMCheckerChain
from langchain.llms import OpenAI

llm = OpenAI()

text = "When is the next Devoxx Belgium event in 2023?"

checker_chain = LLMCheckerChain(llm=llm, verbose=True)

checker_chain.run(text)



> Entering new LLMCheckerChain chain...


> Entering new SequentialChain chain...

> Finished chain.

> Finished chain.


' The next Devoxx Belgium event is scheduled to take place in 2022, not in 2023.'